

```
# 코드로 형식 지정됨
```

## 3.회귀 모델 성능평가

- $R^2$ score
  - model.score(X, y)
  - sklearn.metrics.r2_score(y_true, y_pred)
  - 분산 기반으로 예측 성능을 평가
  - 예측값 분산 / 실제값 분산
- MAE(Mean Absolute Error)
   - sklearn.metrics.mean_absolute_error(y_true, y_pred)
   - 실제 값과 예측 값의 차이를 절댓값으로 변환해 평균한 것
- MSE(Mean Squared Error)
   - sklearn.metrics.mean_squared_error(y_true, y_pred)  
   - 실제 값과 예측 값의 차이를 제곱해 평균한 것
- RMSE(Root Mean Squared Error)
   - sklearn API에 없음
   - MSE에 np.sqrt() 또는 ** 0.5를 사용함
   - MSE 값은 오류의 제곱을 구하므로 실제 오류 평균보다 더 커지는 특성이 있어 MSE에 루트를 씌운 것
- MSLE(Mean Squared Log Error)
   - sklearn.metrics.mean_squared_log_error(y_true, y_pred)
   - MSE에 로그를 적용해준 지표
- RMSLE(Root Mean Square Log Error)
   - RMSE에 로그를 적용해준 지표
   - 이상치가 있더라도 변동폭이 크지 않음(이상치에 강건함)
   - 실제값보다 예측값이 작을 때 더 큰 패널티 부여
   - 작게 예측하면 안되는 경우 사용하면 좋음 (배달 시간 예측)
   - 실제값, 예측값에 음수가 있으면 안됨 (오류 발생)
- 참조 : https://bkshin.tistory.com/entry/%EB%A8%B8%EC%8B%A0%EB%9F%AC%EB%8B%9D-17-%ED%9A%8C%EA%B7%80-%ED%8F%89%EA%B0%80-%EC%A7%80%ED%91%9C

In [54]:
# R2_score : 1에 가까울 수록 좋은 성능
# MAE, MSE, RMSE, MSLE, RMSLE :  0에 가까울 수록 좋은 성능

### 3-1.성능평가 함수

In [55]:
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error as mae
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_squared_log_error as msle

# [29] r2_score, mae, mse, rmse, msle, rmsle 를 구해 반환하는 함수를 작성합니다.
# 한 개의 행에 6가지 성능평가 결과를 소수점 아래 4자리까지 표시되도록 하여 작성합니다.

def get_scores(model,X,Y):
  pred = model.predict(X)

  #음수가 있어도 상관없음
  A = r2_score(Y,pred)
  B = mae(Y,pred)
  C = mse(Y,pred)
  E = np.sqrt(C) #rmse

  # msle,rmsle 는 음수가 있어서는 안됨
  # pred[0] = -2
  pred = np.where(pred<0,0,pred)
  D = msle(Y,pred)
  F = np.sqrt(D) # rmsle
  return f'r2: {A:.4} mae:{B:.4f} mse: {C:.4f} msle: {D:.4f} rmse: {E:.4} rmsle:{D:.4f}'


### 3-2.모델링 함수

In [56]:
# [30] Model_Train함수를 수정해서
# train, test에 대한 6가지 성능평가 결과를 출력하는 Model_Train2 함수를 작성합니다.

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

def Model_Train_2(model,XF,YF,scale=False):
  # 7:3 분리 , random_state=321 (주의: stratify 사용 안함 -> 분류 모델에서만 사용)
  if scale:
    XF = MinMaxScaler().fit_transform(XF)
  xtrain,xtest,ytrain,ytest = train_test_split(XF,YF,test_size=0.3,random_state=321)

  model.fit(xtrain,ytrain)
  print(model.__class__)
  print('train : ',get_scores(model,xtrain,ytrain))
  print('test : ', get_scores(model,xtest,ytest))
  print()
  return model

### 3-3.여러 모델 적용

In [57]:
# [31] Xnormal, Ynormal 데이터 사용, LinearRegression => model1
model1 = LinearRegression()
Model_Train_2(model1,Xnormal,Ynormal)

<class 'sklearn.linear_model._base.LinearRegression'>
train :  r2: 0.9931 mae:0.2649 mse: 0.1198 msle: 0.0003 rmse: 0.3461 rmsle:0.0003
test :  r2: 0.9622 mae:0.3807 mse: 0.3659 msle: 0.0010 rmse: 0.6049 rmsle:0.0010



LinearRegression()

**종합 분석**:
*   훈련 데이터 성능: 훈련 데이터에서 매우 높은 성능을 보이고 있으며, 모델이 훈련 데이터에 거의 완벽하게 맞추고 있다는 것을 나타냅니다.

*   테스트 데이터 성능: 테스트 데이터에서도
여전히 좋은 성능을 보이지만, 훈련 데이터에 비해 성능이 다소 떨어지는 것을 볼 수 있습니다. 특히 MAE, MSE, RMSE가 모두 훈련 데이터보다 상당히 큰 값으로 나타나고 있어, 모델이 테스트 데이터에 대해 약간의 과적합을 일으켰을 가능성이 있습니다.

**결론:**
*   이 모델은 훈련 데이터에서는 매우 잘 학습되었지만, 테스트 데이터에서는 성능이 다소 떨어지는 경향이 있습니다. 이는 **과적합(overfitting)**의 징후일 수 있습니다.

*   추가적인 일반화 작업(예: 정규화, 더 많은 데이터를 사용한 학습, 규제(regularization) 적용)이 필요할 수 있습니다.

In [58]:
# [32] Xpoly3, Ynormal 데이터 사용, LinearRegression => model2
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression

poly3 = PolynomialFeatures(degree=3,include_bias=False)
Xpoly3 = poly3.fit_transform(Xnormal)
model2 = LinearRegression()
print(Xpoly3.shape)
Model_Train_2(model2,Xpoly3,Ynormal)

(100, 9)
<class 'sklearn.linear_model._base.LinearRegression'>
train :  r2: 1.0 mae:0.0025 mse: 0.0000 msle: 0.0000 rmse: 0.003137 rmsle:0.0000
test :  r2: 1.0 mae:0.0077 mse: 0.0004 msle: 0.0000 rmse: 0.01986 rmsle:0.0000



LinearRegression()

In [59]:
# [33] Xlarge, Ylarge 데이터 사용, XGBRegressor => model3

from xgboost import XGBRegressor

dataL = make_sample(1234,2000)
Xlarge = dataL[['height','weight']]
Ylarge = dataL['BMI']
model3 = XGBRegressor(n_estimators=900,objective='reg:squarederror',random_state=0)
Model_Train_2(model3,Xlarge,Ylarge)

<class 'xgboost.sklearn.XGBRegressor'>
train :  r2: 1.0 mae:0.0038 mse: 0.0001 msle: 0.0000 rmse: 0.007885 rmsle:0.0000
test :  r2: 0.9952 mae:0.1564 mse: 0.0669 msle: 0.0002 rmse: 0.2587 rmsle:0.0002



XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=900, n_jobs=None,
             num_parallel_tree=None, random_state=0, ...)

In [60]:
# [34] X_submission, Y_hidden 데이터를 사용하여 model1, model2, model3의 성능을 평가하여봄
XF = X_submission[['height','weight']]
YF = Y_hidden['BMI']
XFpoly3 = poly3.fit_transform(XF)
print(get_scores(model1,XF,YF))
print(get_scores(model2,XFpoly3,YF))
print(get_scores(model3,XF,YF))

r2: 0.9892 mae:0.2705 mse: 0.1503 msle: 0.0004 rmse: 0.3876 rmsle:0.0004
r2: 1.0 mae:0.0037 mse: 0.0000 msle: 0.0000 rmse: 0.005067 rmsle:0.0000
r2: 0.9943 mae:0.1661 mse: 0.0798 msle: 0.0001 rmse: 0.2824 rmsle:0.0001


In [61]:
# [35] 제출파일 만들기 (실제 문제에서 요구하는 형태로 작성해야 함)
# msle, rmsle (음수가 있으면 안됨)
pred = model2.predict(Xpoly3)
submission = pd.DataFrame({'ID': X_submission['ID'],
                           'BMI' : pred})
submission.to_csv('0001100.csv',index=False)